In [1]:
#!pip install obspy
import sys
!{sys.executable} -m pip install cartopy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!{sys.executable} -m pip install mpl_toolkits

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement mpl_toolkits (from versions: none)
ERROR: No matching distribution found for mpl_toolkits


In [3]:
#Run this first
import obspy
import numpy as np
from obspy.clients.fdsn.client import Client
 ##TODOli Can't install cartopy on Noah's local machine
import pandas as pd
import geopandas as gpd
from collections import defaultdict

import matplotlib.cm as cm
from matplotlib.pyplot import figure, show, rc
from matplotlib import pyplot as plt

import plotly.express as px
import cartopy
import cartopy.crs as ccrs                   # import projections
import cartopy.feature as cf 
from cartopy.feature import NaturalEarthFeature
# import Basemap
# from mpl_toolkits.basemap import Basemap


In [4]:
#run this cell to define functions

def quakeML_Loader(filepath): #Loads quakeML file formats as Catalog
    # Make an obspy Catalog object from the QuakeML file.
    return obspy.core.event.read_events(filepath)
    #Copied from Maleen's code
    #Not sure this 1-line function actually needs to exist

def generate_summary(cat): #function that generates a summary table of data from a catalog of events for identification of best candidate events
    resource_ids = []
    magnitudes = []
    pick_counts =[]
    epi_lats = []
    epi_longs = []
    times = []
    depths = []
    for event in cat:
        resource_ids.append(event.resource_id)
        magnitudes.append(event.magnitudes[0].mag)
        pick_counts.append(len(event.picks))
        epi_lats.append(event.origins[0].latitude)
        epi_longs.append(event.origins[0].longitude)
        times.append(event.origins[0].time)
        depths.append(event.origins[0].depth)
    summary = pd.DataFrame(resource_ids, magnitudes).reset_index()
    summary.columns = ['magnitudes','resource_ids']
    summary['pick_counts'] = pick_counts #unsure why this needs to be done on its own but it works
    summary['epi_lats'] = epi_lats
    summary['epi_longs'] = epi_longs
    summary['times'] = times
    summary['depths'] = depths
    #these two lines can be edited for legibility and helpfulness
    print('Biggest earthquake was', summary.sort_values(by='magnitudes', ascending = False).get('resource_ids').iloc[0],'with magnitude',str(summary.sort_values(by='magnitudes', ascending = False).get('magnitudes').iloc[0])+'.')
    print('Best picked earthquake was', summary.sort_values(by='pick_counts', ascending = False).get('resource_ids').iloc[0],'with',summary.sort_values(by='pick_counts', ascending = False).get('pick_counts').iloc[0],'picks.')
    #TODO get index position(s) of most relevant quakes, but for our data set it's i = 170
    return summary

def quakeML_Reader(event):
    epicenter = (event.origins[0].latitude, event.origins[0].longitude) #stores event epicenter lat/lon as tuple
    magnitude = event.magnitudes[0].mag #stores event magnitude
    birthday = event.origins[0].time #stores time at which event occurs
    results = []
    for arrival in event.origins[0].arrivals: #goes through arrival data and notes phase, azimuthal angle, distance from epicenter and pick_id in 2D list
        phase = arrival.phase
        azi = arrival.azimuth
        dist = arrival.distance * 111 #there is a note in Maleen's code about this being in degrees
        pick_id = arrival.pick_id
        result = [phase, azi, dist, pick_id]
        results.append(result)
    arrivals = pd.DataFrame(results)
    arrivals.columns = ['phase','azimuth','distance','pick_id'] #makes DataFrame of arrivals data
    results = []
    for pick in event.picks: #goes through pick data and notes time of arrival, station data and pick_id in 2D list
        pick_id = pick.resource_id
        time = pick.time
        network_code = pick.waveform_id.network_code
        station_code = pick.waveform_id.station_code
        channel_code =pick.waveform_id.channel_code
        result = [pick_id, time, network_code, station_code, channel_code]
        results.append(result)
    picks = pd.DataFrame(results)
    picks.columns = ['pick_id','time','network_code','station_code','channel_code'] #makes DataFrame of picks data
    picks['travel_time'] = picks['time'] - birthday
    log = arrivals.merge(picks, left_on='pick_id', right_on='pick_id').sort_values(by='travel_time') #merges arrivals and picks data
    log['velocity'] = log['distance'] / log['travel_time']
    bonus = (epicenter, magnitude, birthday) #TODO integrate bonus

    #adds lat lon to picks
    #code borrowed from Zoe's explore_data.ipynb
    sta_list = np.unique(log.station_code)

    # Get all the info for those stations from IRIS
    network = ",".join((np.unique(log.network_code)).tolist())
    channel = ",".join((np.unique(log.channel_code)).tolist())
    station = ",".join((np.unique(log.station_code)).tolist())

    starttime = np.min(log['time'])
    endtime = np.max(log['time'])

    sta_metadata = Client("iris").get_stations(starttime=starttime,endtime=endtime,network=network,channel=channel,station=station,location='',level='response')

    station_locs = defaultdict(dict)
    for network in sta_metadata:
        for station in network:
            for chn in station:
                sid = f"{network.code}.{station.code}.{chn.location_code}.{chn.code[:-1]}" + chn.start_date.strftime('%Y%j')
                if sid in station_locs:
                    station_locs[sid]["component"] += f",{chn.code[-1]}"
                    station_locs[sid]["response"] += f",{chn.response.instrument_sensitivity.value:.2f}"
                else:
                    component = f"{chn.code[-1]}"
                    response = f"{chn.response.instrument_sensitivity.value:.2f}"
                    dtype = chn.response.instrument_sensitivity.input_units.lower()
                    tmp_dict = {}
                    tmp_dict["longitude"], tmp_dict["latitude"], tmp_dict["elevation(m)"] = (
                        chn.longitude,
                        chn.latitude,
                        chn.elevation,
                    )
                    tmp_dict["component"], tmp_dict["response"], tmp_dict["unit"] = component, response, dtype
                    tmp_dict["start_date"], tmp_dict["end_date"] = chn.start_date,chn.end_date
                    tmp_dict["network"], tmp_dict["station"] = network.code, station.code
                    station_locs[sid] = tmp_dict

    station_locs = pd.DataFrame.from_dict(station_locs,orient='index')
    station_locs["id"] = station_locs.index
    # Remove the date from ID
    station_locs['id']=station_locs['id'].str.slice(stop=-7)
    loc_log = log.merge(station_locs, left_on='station_code',right_on='station').drop(columns=['component','response','unit','start_date','end_date','id','network_code','station_code'])
    return loc_log

In [ ]:
cat = quakeML_Loader('XO_2019_01.quakeml') #loads quakeML file as events Catalog

#Should take about 1 minute to run on Noah's laptop, be patient

In [ ]:
#generate_summary(cat) #summarizes events and identifies most relevant
# generate_summary(cat)
df = generate_summary(cat)
df.head()

In [ ]:
# df['year'] = pd.DatetimeIndex(df['times']).year # add year as a category to group by
df['marker_size'] =np.fix(np.exp(df['magnitudes'])) # add marker size as exp(mag)
df['magnitude bin'] = 0.5*np.fix(2*df['magnitudes']) # add marker size as exp(mag)
df['location'] = 'Alaskan Coast'

In [ ]:
#Location of earthquakes
fig = px.scatter_geo(df,
                     lat='epi_lats',lon='epi_longs', 
                     size='marker_size', color='magnitudes',
                     hover_name='location', 
                     hover_data=['magnitudes','depths'],
                     scope='north america',
                     range_color=(min(df['magnitudes']),max(df['magnitudes'])),
                     height=600, width=1000,
                     #animation_frame="year"
                    );

# focus point
# lat_foc = 55
# lon_foc = -165
lat_foc = 63
lon_foc = -175
fig.update_layout(
        geo = dict(
            projection_scale=3, #this is kind of like zoom
            center=dict(lat=lat_foc, lon=lon_foc), # this will center on the point
        ))

fig.update_geos(resolution=50, showcountries=True)

In [ ]:
event = cat[170] #biggest and most documented quake
log = quakeML_Reader(event) #be patient, this should take like 10s
log

In [ ]:
fig = px.scatter_geo(log,
                     lat='latitude',lon='longitude', 
                     # size='marker_size', color='magnitudes',
                     hover_name='station', 
                     hover_data=['distance','velocity', 'travel_time'],
                     scope='north america',
                     # range_color=(min(df['magnitudes']),max(df['magnitudes'])),
                     height=600, width=1000,
                     #animation_frame="year"
                    );

# focus point
# lat_foc = 55
# lon_foc = -165
lat_foc = 63
lon_foc = -175
fig.update_layout(
        geo = dict(
            projection_scale=3, #this is kind of like zoom
            center=dict(lat=lat_foc, lon=lon_foc), # this will center on the point
        ))

fig.update_geos(resolution=50, showcountries=True)